In [2]:
##
import os
import glob
import numpy as np
from tqdm import tqdm

In [3]:
def read_train(path):
    with open(path, 'r') as file:
        lines = file.readlines()
    return lines

# Count alignments of target tokens conditioned on each source token
def count_align(src_list, tgt_list, align_list, align_dict):
    for al in align_list:
        als, alt = al.split('-')
        als, alt = int(als), int(alt)
        s_tok, t_tok = src_list[als], tgt_list[alt]
        if s_tok not in align_dict.keys():
            align_dict[s_tok] = dict()
        if t_tok not in align_dict[s_tok].keys():
            align_dict[s_tok][t_tok] = 1
        else:
            align_dict[s_tok][t_tok] += 1
    return align_dict

# Compute uncertainty by the align_dict
def comp_uncertainty(align_dict):
    H = []
    for k,v in align_dict.items():
        tot_k = sum(v.values())    # total counts of alignment to word k
        h = 0
        for kk,vv in v.items():
            p = vv / float(tot_k)  # p for k-kk alignment
            h = h - p * np.log(p)
        H.append(h)
    return H

In [4]:
disk = "/apdcephfs/share_916081/joelwxjiao"
align_path = disk + "/significance-test/linguistic"
data_path = "/wmt14_en_de_base_reju"

In [13]:
# Readin data
path_src = align_path + data_path + "/train_active.en"
path_tgt = align_path + data_path + "/train_active.de"
path_ali = align_path + data_path + "/train_active.alignment"

src_data = read_train(path_src)
tgt_data = read_train(path_tgt)
ali_data = read_train(path_ali)
print(len(src_data), len(tgt_data), len(ali_data))

4021956 4021956 4021956


In [14]:
align_dict = dict()
for idx in tqdm(range(len(src_data))):
    src = src_data[idx].strip('\n').split()
    tgt = tgt_data[idx].strip('\n').split()
    ali = ali_data[idx].strip('\n').split()
    align_dict = count_align(src, tgt, ali, align_dict)

print(len(align_dict))

H = comp_uncertainty(align_dict)

print(np.mean(H), H[:10])

100%|██████████| 4021956/4021956 [02:44<00:00, 24479.77it/s]


32611
2.005513752429398 [2.344597574823132, 2.4896231390260826, 3.78130818645039, 4.154762887225315, 3.128340391782482, 3.69410653646968, 3.9163537121584553, 3.627337110539781, 1.5973391948050726, 3.556133259237476]
